<img src='https://repository-images.githubusercontent.com/121802384/c355bb80-7d42-11e9-9e0e-4729609f9fbc' alt='WRF-Hydro Logo' width="15%"/>

# Lesson 2 - Running WRF-Hydro

## Overview
In this lesson, we cover the basics of constructing and running a WRF-Hydro simulation using a prepared domain for the *gridded* routing configuration. 

For a detailed description of model configurations see the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide). 

## Constructing a simulation with a prepared domain
In this section we describe the primary files needed to run a WRF-Hydro simulation. 

A WRF-Hydro simulation consists of the following major components:
-  executable/binary file
-  parameter files
-  domain files
-  forcing files
-  namelists

In this lesson we only cover basic descriptions of these elements. For a detailed description see the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide).

The WRF-Hydro model code, compiled executable/binary, and associated parameter table files were described in [Lesson 1](Lesson-1-compile.ipynb). 

**If you have not already completed Lesson 1, please stop and do so now**

Model run-time options are specified in two namelist files `hydro.namelist` and `namelist.hrldas`. These namelist files contain file path specifications, simulation duration, physics options, and output file selections, among others. We will cover many of these options in more depth in Lessons 4 and 5.   

## Orientation to the *example_case*
This lesson will use a prepared domain located in the `~/wrf-hydro-training/example_case` directory. The structure of the `example_case` directory serves as a good example of how to organize your domain files. If using another domain with this lesson, such as one you may have created in the geospatial pre-processing tutorial, it is **imperative** that the file names and directory structure match those described below.

The example case chosen for this training is a cutout of the Iceland domain with the gage 19372 as the outlet location. Ther priod we are going to simulate is 2017-09-20 up to 2017-09-30. This time period was selected since there is a high streamflow event observed in the time period, and 10 days simulation is short enough to run quickly and look at the outputs. 

<img src="images/Iceland.png" width="300"/>


Now lets take a look at the example_case directory and review the content of the files that are provided. 

In [ ]:
cd ~/wrf-hydro-training/example_case
ls 

In this example case we have created only one configurations for WRF-Hydro: *Gridded*. For detailed descriptions of routing and physics configurations see the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide). If you look into the official test cases for the WRF-Hydro, there will be multiple configuration for the test case. These model configurations have different geospatial data associated with them, and thus are represented by separate directories in the `~/wrf-hydro-training/example_case` directory. However, in this training module we are showcasing only the *Gridded* configuration which is selected for the Iceland. 

### FORCING
This directory contains all of the forcing data for our simulation. Note that there is only one `FORCING` directory. The same forcing data can be used with all three configurations.

### The Gridded configuration directory
For this lesson, we will be running the *Gridded* configuration. Now we will explore the `~/wrf-hydro-training/example_case/Gridded` directory.

In [ ]:
ls Gridded

For a more detailed description of these files beyond what will be described below, see the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide)

**DOMAIN**: Directory containing all geospatial data and input files for
the Gridded routing option with lakes included

In [ ]:
ls Gridded/DOMAIN

| Filename | Description | Source | Required |
| ------------- | ------------- | ------------- | ------------- |
| Fulldom_hires.nc | High resolution full domain file. Includes all fields specified on the routing grid. | WRF-Hydro GIS pre-processing toolkit | Yes |
| GWBASINS.nc | 2D file defining the locations of groundwater basins on a grid | WRF-Hydro GIS pre-processing toolkit | When the baseflow bucket model is turned on and user defined mapping is off |
| GWBUCKPARM.nc | Groundwater parameter table containing bucket model parameters for each basin | WRF-Hydro GIS pre-processing toolkit | When the baseflow bucket model is turned on |
| LAKEPARM.nc | Lake parameter table containing lake model parameters for each catchment | WRF-Hydro GIS pre-processing toolkit | When lake and reservoir routing is turned on |
| hydro2dtbl.nc | Spatially distributed parameter table for lateral flow routing within WRF-Hydro. | create_SoilProperties.R script (will also be automatically generated by WRF-Hydro) | When using spatially distributed terrain routing parameters |
| geo_em.d01.nc | The data required to define the domain and geospatial attributes of a spatially-distributed, or gridded, 1-dimensional (vertical) land surface model (LSM) | GEOGRID utility in the WRF preprocessing system (WPS) | Yes |
| wrfinput_d01.nc | file including all necessary fields for the Noah-MP land surface model, but with spatially uniform initial conditions. Users should be aware that the model will likely require additional spin-up time when initialized from this file. | create_Wrfinput.R script | Yes |
| soil_properties.nc | Spatially distributed land surface model parameters | create_SoilProperties.R script | If SPATIAL_SOIL compile-time option set to 1 |
| GEOGRID_LDASOUT_Spatial_Metadata.nc | projection and coordinate information for the land surface model grid. | pre-processor | No, but allows for CF compliant outputs |
| lake_shapes/ | supplemental shape files that define lakes | pre-processor | No |



Note the list above does not have a lakeparm file, since the domain used in this training is above lakes and therefore there is no lake in the domain. If you refer to the main domain files for the Iceland, there will be a LAKEPARM.nc file which has all the properties of the lakes in them. 

**RESTART**: Directory containing model restart files

In [ ]:
ls Gridded/RESTART

Restart files are an essential part of the WRF-Hydro modeling system. They are output on a fixed timestep specified by the user in the `namelist.hrldas` and `hydro.namelist` files, and represent a complete 'snapshot' of the model state at that time. These files can be used to restart a WRF-Hydro simulation from where the previous simulation terminated with all the model states intact.

When running a WRF-Hydro simulation, you may start your simulation with default initial conditions, referred to as a 'cold start'. When starting from a cold start, a model spinup period is needed to move the model state away from the default initial conditions to a more realistic, physically-based model state. Model output from the spinup period is generally not used for interpretation.

Restart files output at the end the spinup period can be used as the initial conditions for subsequent simulations, referred to as a 'warm start'. Simulations that start from a 'warm start' are generally the primary target for interpretation. 'Warm' and 'cold' starting the model will be discussed more in Lesson 4. 

### Namelists
Namelists are another key component of the WRF-Hydro modeling system and are the primary means of specifying inputs, outputs, and run-time options. There are two namelist files used by WRF-Hydro: `hydro.namelist` for the routing and hydrologic model and `namelist.hrldas` for the land surface model. 

**NOTE: These filenames are hard-coded into the model and can not be changed.**

For all official WRF-Hydro domains, namelists will be included with each of the model configurations. This is done so that a user can easily run each configuration with minimal setup, and they serve as a starting point for users to specify their own namelists for a given configuration.

Take a moment and read through the two namelists below and note how all filepaths are relative to the current directory containing the namelist. This is the recommended way of specifying filepaths.

In [ ]:
cat Gridded/hydro.namelist

In [ ]:
cat Gridded/namelist.hrldas

## Creating a simulation directory 
Now that we have covered the major functional elements that constitute a simulation, we will combine these elements and construct a simulation. This is done by placing the `FORCING`, `Gridded/DOMAIN`, `trunk/NDHMS/Run` directories and `namelist.hrldas` and `hydro.namelist` files together in a directory that will be our simulation directory. However, to save disk space it is often preferable to create symbolic links rather than copying the actual files. 

**NOTE: We will only use symbolic links with files that we will NOT be editing**    

![fig2.png](images/fig2.png)

In the following steps, we will construct our simulation directory.

**Step 1. Create simulation directory**

We will create a directory for our simulation

In [ ]:
mkdir -p ~/wrf-hydro-training/output/lesson2/run_gridded_default
cd ~/wrf-hydro-training/output/lesson2/run_gridded_default

**Step 2. Copy model run files**

We will copy the required model run files from the *~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run* directory. These files are small so we will make actual copies rather than symbolic links in this case. Additionally, copies are preferred in this case because a user may want to edit the \*.TBL files and as stated previously symbolic links should not be used with files that we may edit.

In [ ]:
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL .
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe .

ls 

The default value of SWE_Limit in the MPTABLE.TBL is 5 meter which can be reached easily in glacier domains. If we exceed the max SWE limit, then all the precipitation becomes runoff. Let's edit the MPTABLE and set the SWE_LIMIT to a high number. 

In [ ]:
%%bash
# Set the SWE Limit to a very high number
sed -i -e 's#SWE_LIMIT = 5000.0#SWE_LIMIT = 5000000.0#g' MPTABLE.TBL

**Step 3. Symlink *DOMAIN* files**

We will create symbolic links to the required *domain* files from the `~/wrf-hydro-training/DOMAIN/Gridded` directory. These files can be large so we will make symbolic links rather than copying the actual files. 

In [ ]:
ln -sf ~/wrf-hydro-training/example_case/FORCING .
ln -sf ~/wrf-hydro-training/example_case/Gridded/DOMAIN .
ln -sf ~/wrf-hydro-training/example_case/Gridded/RESTART .

ls 

**Step 4. Copy *namelist* files**

Because we are using the default prepared namelists from the example WRF-Hydro domain, we will copy those in as well. If you were using your own namelists, they would likely be edited and copied from elsewhere. These are small text files so we will make actual copies rather than symbolic links.

In [ ]:
cp ~/wrf-hydro-training/example_case/Gridded/namelist.hrldas .
cp ~/wrf-hydro-training/example_case/Gridded/hydro.namelist .

ls 

We have now constructed our simulation directory with all the requisite files. 

In the next section we will run the simulation. 

## Running WRF-Hydro using default run-time options
Now that we have constructed our simulation directory, we can run our simulation. For this we will be using the `mpirun` command, which has a number of arguments. For this simple case, we only need to supply one argument, the number of cores. This is done with the `-np` argument, and we will set it to 2 cores. 

We will pipe the output to a log file because running a simulation can generate a lot of standard output in the terminal.

In [ ]:
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

If your simulation ran successfully, there should now be a large number of output files in the `~/wrf-hydro-training/output/lesson2/run_gridded_default`. We will describe these output files in more depth in Lesson 4. Additionally, detailed descriptions of the output files can be found in the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide). 

List the contents of the run directory.

In [ ]:
ls

There are also four important files for determining the success or failure of the run, diag_hydro.0000\*. The number of diag files is equal to the number of cores used for the run. These diag_hydro.0000\* files contain logs and diagnostics on the simulation run. 

You can check that your simulation ran successfully by examining the last line of the diag files, which should read  `The model finished successfully........`

In [ ]:
tail -1 diag_hydro.00000

# Next up - Basics of working with WRF-Hydro outputs
This concludes Lesson 2. In the next lesson, we will **briefly** discuss working with some of the output files. The output files from WRF-Hydro are standard netCDF4 files, and thus there are many way to work with these data. We simply cover a few Python libraries and commands that are needed for later lessons in this tutorial. 

[Lesson 3](Lesson-3-visualize.ipynb) is by no means a comprehensive guide to working with netCDF files. 

**IT IS BEST TO EITHER SHUTDOWN THIS LESSON OR CLOSE IT BEFORE PROCEEDING TO THE NEXT LESSON TO AVOID POSSIBLY EXCEEDING ALLOCATED MEMORY. Shutdown the lesson be either closing the browser tab for the lesson or selecting `Kernel -> Shut Down Kernel` in JupyterLab.**

© UCAR 2020